### Application of Divide and Conquer in multiplying polynomials

- We can apply divide and conquer on polynomial multiplcation to get more efficient product

- Suppose $A(x) = 3x^2 + 2x + 5, B(x) = 5x^2 + x + 2$. Then $A(x) B(x) = 15x^4 + 13x^3 + 33 x^2 + 9x + 10$

- General problems statement
    - **Input:** 2 $n-1$ degree polynomials $A = a_{n-1} x^{n-1} + a_{n-2} x^{n-2} + ... a_1 x + a_0$ and $B = b_{n-1} x^{n-1} + b_{n-2} x^{n-2} + ... b_1 x + b_0$
    - **Output:** The product polynomial $C = c_{2n-2} x^{2n-2} + c_{2n-3} x^{2n-3} + ... c_1 x + c_0$. Note the following relation between $a$, $b$ and $c$ coefficients:
        - $c_{2n-2} = a_{n-1} \cdot b_{n-1}$
        - $c_{2n-3} = a_{n-1} \cdot b_{n-2} + a_{n-2} \cdot b_{n-1}$ 
        - $c_{2n-4} = a_{n-1} \cdot b_{n-3} + a_{n-2} \cdot b_{n-2} + a_{n-3} \cdot b_{n-1}$ 
        - $...$
        - $c_2 = a_{2} \cdot b_{0} + a_{1} \cdot b_{1} + a_{0} \cdot b_{2}$
        - $c_1 = a_{1} \cdot b_{0} + a_{0} \cdot b_{1}$
        - $c_0 = a_{0} \cdot b_{0}$

- Rewriting the previous example
    - **Input:** n=3, A = (3,2,5), B=(5,1,2)
    - **Output:** C = (15, 13, 33, 9, 10)     

- Naive solution
    - To solve this, we can iterate through the coefficients in arrays $A$ and $B$. For every pair, we compute the value, and add it into the appropriate slot for $c$
    - This is $O(N^2)$, so it's obviously bad

In [1]:
def naive_mult_poly(A, B, n):
    '''
    time complexity: O(N^2) because of nested for loop
    space complexity: O(N)
    '''
    product = [0] * ((2*n)-1)
    for i in range(n-1):
        for j in range(n-1):
            product[i+j] += product[i+j] + (A[i] * B[j])
    return product


- Naive divide and conquer
    - Surprisingly, there is a way to apply divide and conquer to solve this problem, though it isn't straightforward. 
    - Let's first try a naive divide and conquer approach
    
    - Let the 2 polynomials be $A = a_{n-1} x^{n-1} + a_{n-2}x^{n-2} + ... + a_{1}x + a_0$ and $B = b_{n-1}x^{n-1} + b_{n-2}x^{n-2} + ... + b_{1}x + b_0$. 

- To get the recursion, notice that we can rewrite $A = C \cdot x^{\frac{n}{2}} + D$ and $B = E \cdot x^{\frac{n}{2}} + F$ 
    $$A = (a_{n-1} x^{\frac{n}{2} - 1} + a_{n-2} x^{\frac{n}{2} - 2} + ... + a_{n - \frac{n}{2}} x^{\frac{n}{2} - \frac{n}{2}}) \cdot x^{\frac{n}{2}} + (a_{\frac{n}{2} - 1} x^{\frac{n}{2} - 1} + a_{\frac{n}{2} - 2} x^{\frac{n}{2} - 2} + a_{\frac{n}{2} - \frac{n}{2}} x^{\frac{n}{2} - \frac{n}{2}})$$
    - Basically we factor out $x^{\frac{n}{2}}$ from all polynomial terms with $x^a \ge x^{\frac{n}{2}}$
    - Notice that after factorising, the coefficient of $x^{\frac{n}{2}}$ is just another polynomial. We can apply the same logic, and split that into a sum of its own sub polynomials!
    - This iterative halving of the coefficient is the basis of the recursion

- What is the base case of the recursion?
    - This splitting of higher order polynomials will go on until we hit this case; $A = a_1 x + a_0$ and $B = b_1 x + b_0$, where the coefficients are no longer polynomials
    - Multiplying it out, it is obvious that $A \cdot B = a_1 b_1 x^2 + (a_1 b_0 + a_0 b_1) \cdot x + a_0 b_0$
    - So in the base case, we perform 4 operations; $(a_1 b_1, a_1 b_0, a_0 b_1, a_0 b_0)$

- So clearly, at every step, 1 operation becomes 4! At each step, the amount of work is:
    - Step 0: $4^0$
    - Step 1: $4^1$
    - Step 2: $4^2$
    - ...
    - Step n: $4^n$

- How many times can we split the polynomial of degree $n$? 
    - Obviously, since we are halving the power at each step, we can do this for $\log_2(n)$ steps

- Recurrence relation: $T(n) = 4 T(\frac{n}{2}) + kn$
    - 4 because we are doing 4 operations
    - $n/2$ because we are halving the polynomial degree
    - $kn$ because we are taking the sum of $n$ coefficients, multiplied by some constant work $k$

- So the total work done is $\sum_{i=0}^{\log_2(n)} 4^i$, 
    - This is $O(4^{\log_2(n)}) = O(2^{2 \log_2(n)}) = O(2^{\log_2(n^2)}) = O(n^2)$


In [181]:
def zero_pad_list(A: list, desired_len: int, direction: str):
    if len(A) < desired_len:    
        if direction == 'left':
            return [0]*(desired_len-len(A)) + A
        elif direction == 'right':
            return A + [0]*(desired_len-len(A))
        else:
            raise ValueError('Pad direction must be `left` or `right`')
    else:
        return A

def naive_dnc_mult_poly(A: list[str], B: list[str], log_recurs: int = 0, verbose: bool = False):
    indent = ' '*3*log_recurs

    if verbose:
        print('='*50)
        print(indent + f'''Calling {A=} {B=}''')
    
    if (len(A) == 1) and (len(B) == 1):
        if verbose:
            print(indent + f'''Returning {A=} * {B=} = {A[0]*B[0]}''')
        return [int(A[0]) * int(B[0])]
    
    max_len = max(len(A), len(B))
    if max_len % 2 != 0:
        max_len += 1

    if (len(A) < max_len):
        A = zero_pad_list(A, max_len, 'left')
    if (len(B) < max_len):
        B = zero_pad_list(B, max_len, 'left')
        
    if verbose:
        print(indent + f'''Post-padding: {A=} {B=}''')
    
    max_len = max(len(A), len(B))
    halved_len = max_len//2
    if verbose:
        print(indent + f'''{max_len=}, {halved_len=}''')

    a = A[:halved_len]
    b = A[halved_len:]
    c = B[:halved_len]
    d = B[halved_len:]
    if verbose:
        print(indent + f'''Splitting: {a=}, {b=}, {c=}, {d=}''')

    coef_2 = naive_dnc_mult_poly(a, c, log_recurs=log_recurs+1, verbose=verbose)
    coef_2 = zero_pad_list(coef_2, (max_len*2)-1, 'right')

    coef_0 = naive_dnc_mult_poly(b, d, log_recurs=log_recurs+1, verbose=verbose)
    coef_0 = zero_pad_list(coef_0, (max_len*2)-1, 'left')
    
    ad = naive_dnc_mult_poly(a, d, log_recurs=log_recurs+1, verbose=verbose)
    bc = naive_dnc_mult_poly(b, c, log_recurs=log_recurs+1, verbose=verbose)
    coef_1 = [x+y for x,y in zip(ad, bc)]
    pad_left = (((max_len*2)-1) - len(coef_1)) // 2    
    pad_right = max_len - pad_left
    coef_1 = [0]*pad_left + coef_1 + [0]*pad_right
    
    if verbose:
        print(indent + f'''{coef_2=}, {coef_1=}, {coef_0=}''')
    final = [x+y+z for x,y,z in zip(coef_2, coef_1, coef_0)]

    if verbose:
        print(indent + f'''{final=}''')
    return final

A=[1,2]
B=[3,4,5,6]
naive_dnc_mult_poly(A,B,verbose=False)
# print(A * B)

[0, 0, 3, 10, 13, 16, 12]

- So we have taken our original naive solution which was $O(N^2)$, and applied divide and conquer to get...another $O(N^2)$ solution. Not amazing

- But we can be smarter about how we do this! 
    - Recall that we're rewriting polynomials $A$ and $B$ as $A = C x^{\frac{n}{2}} + D$ and $B = E x^{\frac{n}{2}} + F$ 
    - As we discussed above, when multiplying out $A$ and $B$, we're doing 4 multiplications $a_1 b_1, a_1 b_2, a_2 b_1, a_2 b_2$
        - For example, let $A = a_1 x + a_2$ and $B = b_1 x + b_2$. 
        - Then $A \cdot B = a_1 b_1 x^2 + (a_1 b_2 + a_2 b_1) x + a_2 b_2$
    - But do we really need 4 multiplications? 
        - Notice that $(a_1 + a_2) \cdot (b_1 + b_2) = a_1 b_1 + a_2 b_1 + a_1 b_2 + a_2 b_2$
        - So $(a_1 + a_2) \cdot (b_1 + b_2) - a_1 b_1 - a_2 b_2 = a_2 b_1 + a_1 b_2 $
        - So the middle term $a_2 b_1 + a_1 b_2$ can be derived from the product of only 3 multiplications, 2 of which ($a_1 b_1, a_2 b_2$) have already been computed!
        - You just need to do 1 more multiplication $(a_1 + a_2) \cdot (b_1 + b_2)$

- Using this approach, we now have 3 multiplications instead of 4 for every divide and conquer step
    - There are $log_2(n)$ divisions for an input of size $n$
    - At each step $i$, we need to split the existing input into 3 operations, yielding $3^i$ units of work
    - So the total work done is now $\sum_{i=0}^{\log_2(n)} 3^i$
    - This gives us time complexity of 
    $$\begin{aligned}
        O(3^{\log_2(n)}) &= O(n^{\log_2(3)}) \\
        &\approx O(n^{1.585}) 
        \end{aligned}$$
    - Which is faster than our previous $n^2$ solution!
 

- This solution is known as the **Karatsuba approach**, and is applied in the computation of large polynomials! Let's try an implementation

In [191]:
def zero_pad_list(A: list, desired_len: int, direction: str):
    if len(A) < desired_len:    
        if direction == 'left':
            return [0]*(desired_len-len(A)) + A
        elif direction == 'right':
            return A + [0]*(desired_len-len(A))
        else:
            raise ValueError('Pad direction must be `left` or `right`')
    else:
        return A

def karatsuba_mult_poly(A: list[str], B: list[str], log_recurs: int = 0, verbose: bool = False):
    indent = ' '*3*log_recurs

    if verbose:
        print('='*50)
        print(indent + f'''Calling {A=} {B=}''')
    
    if (len(A) == 1) and (len(B) == 1):
        if verbose:
            print(indent + f'''Returning {A=} * {B=} = {A[0]*B[0]}''')
        return [int(A[0]) * int(B[0])]
    
    max_len = max(len(A), len(B))
    if max_len % 2 != 0:
        max_len += 1
    
    if (len(A) < max_len):
        A = zero_pad_list(A, max_len, 'left')
    if (len(B) < max_len):
        B = zero_pad_list(B, max_len, 'left')
        
    if verbose:
        print(indent + f'''Post-padding: {A=} {B=}''')
    
    max_len = max(len(A), len(B))
    halved_len = max_len//2
    if verbose:
        print(indent + f'''{max_len=}, {halved_len=}''')

    a = A[:halved_len]
    b = A[halved_len:]
    c = B[:halved_len]
    d = B[halved_len:]
    if verbose:
        print(indent + f'''Splitting: {a=}, {b=}, {c=}, {d=}''')

    coef_2 = karatsuba_mult_poly(a, c, log_recurs=log_recurs+1, verbose=verbose)
    coef_0 = karatsuba_mult_poly(b, d, log_recurs=log_recurs+1, verbose=verbose)
    a_plus_b = [x+y for x,y in zip(a,b)]
    c_plus_d = [x+y for x,y in zip(c,d)]
    apb_times_cpd = karatsuba_mult_poly(a_plus_b, c_plus_d, log_recurs=log_recurs+1, verbose=verbose)
    coef_1 = [x-y-z for x,y,z in zip(apb_times_cpd, coef_2, coef_0)]

    coef_2 = zero_pad_list(coef_2, (max_len*2)-1, 'right')
    coef_0 = zero_pad_list(coef_0, (max_len*2)-1, 'left')
    pad_left = (((max_len*2)-1) - len(coef_1)) // 2
    pad_right = max_len - pad_left
    coef_1 = [0]*pad_left + coef_1 + [0]*pad_right
    
    if verbose:
        print(indent + f'''{coef_2=}, {coef_1=}, {coef_0=}, {a_plus_b=}, {c_plus_d=}, {apb_times_cpd=}''')
    final = [x+y+z for x,y,z in zip(coef_2, coef_1, coef_0)]

    if verbose:
        print(indent + f'''{final=}''')
    return final

A=[1,2]
B=[3,4,5,6,7]
print(naive_dnc_mult_poly(A,B,verbose=False))
print(karatsuba_mult_poly(A,B,verbose=False))

[0, 0, 0, 0, 0, 0, 3, 15, 24, 19, 14]
[0, 0, 0, 0, 0, 0, 3, 15, 24, 19, 14]


- One application of the Karatsuba algorithm is in multiplying large numbers together quickly. This is a straightforward extension, since any integer can be written as a polynomial where $x = 10$
- The actual implementation when multiplying numbers is different from polynomials; when multiplying numbers, we can simply add up the intermediate values, because actual values won't 

In [ ]:
def zero_pad(A: str, desired_len: int, direction: str):
    if len(A) < desired_len:    
        if direction == 'left':
            return '0'*(desired_len-len(A)) + A
        elif direction == 'right':
            return A + '0'*(desired_len-len(A))
        else:
            raise ValueError('Pad direction must be `left` or `right`')
    else:
        return A

def naive_dnc_mult_numbers(A: str, B: str, verbose=False):
    '''
    Here, we multiply A and B as if they are integers, because having A and B as regular numbers is just a special case of polynomial multiplication when x=10. The result is generalisable to normal polynomial multiplication
    '''
    A = str(A)
    B = str(B)

    if verbose:
        print('='*50)
        print(f'{A=} | {B=}')

    if (len(A) == 1) or (len(B) == 1):
        if verbose:
            print(f'Returning {int(A)} * {int(B)}')
        return str(int(A) * int(B))
        
    max_len = max(len(A), len(B))
    if max_len % 2 != 0:
        max_len += 1    
    A = zero_pad(A, max_len, 'left')
    B = zero_pad(B, max_len, 'left')
    if verbose:
        print(f'Padded: {A=} | {B=}')

    mid_len = max_len // 2

    a = A[:mid_len]
    b = A[mid_len:]
    c = B[:mid_len]
    d = B[mid_len:]
    if verbose:
        print(f'Split: {a=} | {b=} | {c=} | {d=}')

    ac = naive_dnc_mult_poly(int(a), int(c),verbose=verbose)
    bd = naive_dnc_mult_poly(int(b), int(d),verbose=verbose)
    ad = naive_dnc_mult_poly(int(a), int(d),verbose=verbose)
    bc = naive_dnc_mult_poly(int(b), int(c),verbose=verbose)
    if verbose:
        print(ac, bd, ad, bc)

    if verbose:
        print(f"Returning {ac=}, {bd=}, {ad=}, {bc=}")
    return str(int(ac) * 10**max_len + (int(ad) + int(bc)) * 10**mid_len + int(bd))

def karatsuba_mult_numbers(A: str, B: str, verbose=False, log_recurs=0):
    log_recurs += 1
    indent = ' '*3*log_recurs
    A = str(A)
    B = str(B)

    if verbose:
        print(indent + '='*50)
        print(indent + f'{A=} | {B=}')

    if (len(A) == 1) or (len(B) == 1):
        if verbose:
            print(indent + f'Returning {int(A)} * {int(B)}')
        return str(int(A) * int(B))
        
    max_len = max(len(A), len(B))
    if max_len % 2 != 0:
        max_len += 1    
    A = zero_pad(A, max_len, 'left')
    B = zero_pad(B, max_len, 'left')
    if verbose:
        print(indent + f'Padded: {A=} | {B=}')

    mid_len = max_len // 2

    a = A[:mid_len]
    b = A[mid_len:]
    c = B[:mid_len]
    d = B[mid_len:]
    if verbose:
        print(indent + f'Split: {a=} | {b=} | {c=} | {d=}')

    ac = karatsuba_mult_numbers(int(a), int(c),verbose=verbose,log_recurs=log_recurs)
    bd = karatsuba_mult_numbers(int(b), int(d),verbose=verbose,log_recurs=log_recurs)
    
    a_plus_b = str(int(a) + int(b))
    c_plus_d = str(int(c) + int(d))
    apb_cpd = karatsuba_mult_numbers(a_plus_b, c_plus_d,verbose=verbose,log_recurs=log_recurs)
    ad_plus_bc = str(int(apb_cpd) - int(ac) - int(bd))
    if verbose:
        print(indent + f"{ac=}, {bd=}, {a_plus_b=}, {c_plus_d=}, {apb_cpd=}, {ad_plus_bc=}")
        print(indent + f'''Returning {int(ac) * 10**max_len=} + {(int(ad_plus_bc)) * 10**mid_len=} + {int(bd)} = {int(ac) * 10**max_len + (int(ad_plus_bc)) * 10**mid_len + int(bd)}''')
    return str(int(ac) * 10**max_len + (int(ad_plus_bc)) * 10**mid_len + int(bd))

A=12
B=34567
print('+'*50)
print(naive_dnc_mult_poly(A,B,verbose=False))
print('+'*50)
print(karatsuba_mult_numbers(A,B,verbose=False))
print('+'*50)
print(int(A) * int(B))